注意：请在以下单元格中填写您的信息后再运行该程序！

In [ ]:
# 请填写您的用户名和密码
USER_ID = "10001"
USER_PASSWORD = "00000"
# 请填写您的域名地址
BASE_DOMAIN = "http://star-eyes.cloud"

In [ ]:
!pip install deeplabcut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#引入deeplabcut环境并创建DLC项目

%reload_ext numpy
%reload_ext matplotlib
%reload_ext mpl_toolkits
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
PATH = "/content/drive/MyDrive/DLC"
CONF_PATH = "/content/drive/MyDrive/DLC/config.yaml"

try:
  os.mkdir("working_dir")
except FileExistsError:
  pass

# !touch ./working_dir/temp.mp4
import deeplabcut


# video_path = "/content/working_dir/temp.mp4"
# videotype = os.path.splitext(video_path)[-1].lstrip('.')




# path_config_file = deeplabcut.create_pretrained_project(
#   project_name,
#   your_name,
#   [video_path],
#   videotype=videotype,
#   model=model2use,
#   analyzevideo=False,
#   createlabeledvideo=False,
#   copy_videos=True, #must leave copy_videos=True
# )
# os.remove("./working_dir/temp.mp4")
#安装完成后必须通过下列命令重启运行时


# os._exit(0)

In [ ]:
# 从文件夹中读取视频文件名，挑选当前分钟的视频文件，将视频文件送入DEEPLABCUT进行处理
!pip install wget
!pip install shed
!pip install bs4
import re
import os
import datetime
import pytz
import wget
import requests
import shed
from bs4 import BeautifulSoup
beijing = pytz.timezone("Asia/Shanghai")
working_dir = "/content/working_dir/"
csv_dir = "/content/csv_dir/"
def process_main():
  try:
    os.mkdir("/content/working_dir")
  except FileExistsError:
    pass

  try:
    os.mkdir("/content/csv_dir")
  except FileExistsError:
    !rm -rf /content/csv_dir/*
  # working_dir = "E:\\process working dir\\"
  

  
  now = datetime.datetime.now(beijing) - timedelta(minutes=4)  #考虑到时间上传延迟，获取三分钟前的时间
  nowstr = now.strftime('%Y%m%d_%H%M')
  nowstr = nowstr + ".mp4"
  regex = r"\D{0}" + nowstr + r"\D{0}"
  print(regex)

  # TODO:使用list建立已处理文件表，未处理的文件则送入filtered_links list中
  # 从nginx中读取文件列表，存入links中
  url = BASE_DOMAIN + "/vstream/" # assuming this is the nginx file server address
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  links = soup.find_all("a") # find all links in the HTML page
  

  #将links从resultset类型转化为list类型
  links = list(links)
  print(links)
  print(type(links))
  links = [str(link) for link in links]
  # 过滤文件列表
  filtered_links = list(filter(lambda x: re.findall(regex, x), links))
  
  pattern = r"<[^>]*>"
  for i in range(len(filtered_links)) :
    filtered_links[i] = url + re.sub(pattern, "", filtered_links[i])
  print(filtered_links)
  if(len(filtered_links) == 0):
    print("Do not find any videos that should be processed at nginx server. This program will do nothing this minute.")
    return 0;
  # 批量下载
  !rm -rf //content/working_dir/*
  for filtered_link in filtered_links:
    print("Downloading file: " + filtered_link)
    wget.download(filtered_link,out=working_dir)

  # 调用DLC处理
  filtered_files = os.listdir(working_dir)
  for filtered_file in filtered_files:
    print("Processing file: " + filtered_file)
    filtered_file = "/content/working_dir/" + filtered_file
    dlc_process(filtered_file)



  #清空工作文件夹(在每次处理循环最后)
  # for file in os.listdir(working_dir):
  #   os.remove(working_dir + file)
  # 清空working_dir
  !rm -rf /content/working_dir/*



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

Fri Apr 14 15:27:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def dlc_process(video_path):
  video_path = deeplabcut.DownSampleVideo(video_path, width=300)
  print(video_path)
  deeplabcut.analyze_videos(CONF_PATH,[video_path],gputouse=0,save_as_csv=True,destfolder=csv_dir)
  # 调用LSTM分析生成的csv文件
  csv_list = os.listdir(csv_dir)
  # 筛选以.csv结尾的文件
  csv_list = [s for s in csv_list if s.endswith(".csv")]
  # 将csv转交给LSTM
  for csv_file in csv_list:
    csv_file_fullpath = csv_dir + csv_file
    print(csv_file_fullpath)
    LSTM_Predict(csv_file_fullpath)
    !rm -rf ${csv_file_fullpath}    #在处理完后清理该文件




In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import datetime
from sklearn.preprocessing import MinMaxScaler
#mydata[['x1','x2','x3']] = MinMaxScaler().fit_transform(mydata[['x1','x2','x3']])


FRAME_NUMBER = 72
def LSTM_Predict(csv_file_fullpath):
  print("predicting:")
  print(csv_file_fullpath)
  #读入csv至numpy
  pd_tmp = pd.read_csv(csv_file_fullpath,header=1,usecols=lambda x:x!="bodyparts")
  pd_tmp = pd_tmp.drop(index = 0)
  pd_tmp = pd_tmp.astype(float)
  numpy_origin = pd_tmp.to_numpy()
  # numpy_origin = np.loadtxt(csv_file_fullpath,delimiter=",")
  print(numpy_origin)
  print(numpy_origin.shape)
 
  #切割900帧（一分钟）的numpy array,LSTM一次处理72帧
  CUT_NUM = int(numpy_origin.shape[0] - numpy_origin.shape[0] % 72) / 72
  # numpy_origin = np.pad(numpy_origin, ((0, pad_rows), (0, 0)), 'constant')
  numpy_origin = np.delete(numpy_origin,range(int(CUT_NUM*72),numpy_origin.shape[0]),axis=0)
  numpy_aftercut = numpy_origin.reshape(int(CUT_NUM),72, 60)
  # numpy_aftercut = numpy_aftercut[:,:,:-1]
  print("after cut:")
  print(numpy_aftercut)
  print(numpy_aftercut.shape)
  for i in range (numpy_aftercut.shape[0]):
    numpy_iter =  numpy_aftercut[i,:,:]
    if (likelihood_check(numpy_iter)==True):
      print("have animal, analyzing with LSTM")
      LSTM_classification(numpy_iter,csv_file_fullpath)
    else :
      print("no animal")
  


#判断csv中有无动物；若有动物则调用model predict,若无直接退出，不写入数据库，likelihood阈值设为0.07
def likelihood_check(numpy_given):
  likelihood_sub_numpy = numpy_given[:,2::3]
  print("Likelihood numpy:")
  print(likelihood_sub_numpy)
  print(likelihood_sub_numpy.dtype)
  average = np.mean(likelihood_sub_numpy)
  print(average)
  if(average<0.07):
    print("There is no animal in this piece of video.")
    return False
  return True

def LSTM_classification(numpy_iter,csv_file_fullpath):
  #x,y坐标处理 & 归一化数据
  scaler = MinMaxScaler()
  flag = True;
  for j in range(72):
    x_cor_tmp = numpy_iter[j][::3]
    y_cor_tmp = numpy_iter[j][1::3]
    x_cor_tmp = scaler.fit_transform(x_cor_tmp.reshape(-1,1))
    y_cor_tmp = scaler.fit_transform(y_cor_tmp.reshape(-1,1))
    if(flag): #create a new np array
      flag = False;
      np_after_scaled =np.concatenate([x_cor_tmp.flatten(),y_cor_tmp.flatten()],axis = 0)
    else: 
      np_after_scaled = np.vstack([np_after_scaled,np.concatenate([x_cor_tmp.flatten(),y_cor_tmp.flatten()],axis = 0)])
  np_after_scaled = np_after_scaled.reshape(1,72,40)
  print(np_after_scaled.shape)

  #加载LSTM模型
  LSTM_model = tf.keras.models.load_model("/content/drive/MyDrive/LSTM_model")
  model_output = LSTM_model.predict(np_after_scaled)
  print(model_output)

  #模型输出：0/eating 2/jumping 3/walking
  print(model_output[0][0])
  # 设定事件类型
  if(max(model_output[0])<0.6):
    event_type = "发现动物出现"
  elif(model_output[0][0]<max(model_output[0][1],model_output[0][2])):
    event_type = "发现动物正在进食"
  elif(model_output[0][1]<max(model_output[0][2],model_output[0][0])):
    event_type = "发现动物正在跳跃"
  else:
    event_type = "发现动物正在移动"
  
  # 设定事件发生时间
  event_time = datetime.datetime.now(beijing) - datetime.timedelta(minutes=4)
  event_time = event_time.strftime("%Y-%m-%d %H:%M")
  cam_id = re.split("_",csv_file_fullpath)
  print(cam_id)
  cam_id = re.split("/",cam_id[0])
  cam_id_final = cam_id[-1]
  print(type(cam_id_final))
  print(cam_id_final)
  print(event_time)
  print(event_type)
  event_class = Event(cam_id=cam_id_final,time =event_time,event_type = event_type)
  update_sql(event_class)
  



In [ ]:
# 事件class
class Event:
  cam_id = "00000"
  time = ""
  event_type = "default"
  def __init__(self,cam_id,time,event_type):
    self.cam_id = cam_id
    self.time = time
    self.event_type = event_type

  def print_self():
    print(cam_id)
    print(time)
    print(event_type)


In [ ]:
def update_sql(event_to_update):
  API_URL = "http://star-eyes.cloud:9090/api/event/addEvent/"
  headers = {'Content-Type': 'application/json'}
  print(API_URL)
  print(USER_ID)
  data = {"user_id":USER_ID,"password":USER_PASSWORD,"cam_id":event_to_update.cam_id,"time":event_to_update.time,"event_type":event_to_update.event_type}
  print(data)
  response = requests.post(url=API_URL, headers=headers,data = data)
  print(response.status_code)
  print(response.text)


In [ ]:
#设定定时器，每隔一分钟执行一次main_process()
import sched
import time
from datetime import timedelta

s = sched.scheduler(time.time, time.sleep)

def run():
  process_main() # 执行函数
  s.enter(60, 1, run) # 每隔 60 秒执行一次函数

s.enter(0, 1, run) # 第一次执行
s.run() # 启动调度器

\D{0}20230321_1900.mp4\D{0}
[<a href="../">../</a>, <a href="61206_20230321_1617.mp4">61206_20230321_1617.mp4</a>, <a href="61206_20230321_1856.mp4">61206_20230321_1856.mp4</a>, <a href="61206_20230321_1857.mp4">61206_20230321_1857.mp4</a>, <a href="61206_20230321_1858.mp4">61206_20230321_1858.mp4</a>, <a href="61206_20230321_1900.mp4">61206_20230321_1900.mp4</a>]
<class 'list'>
['http://star-eyes.cloud/vstream/61206_20230321_1900.mp4']
Processing file: 61206_20230321_1900.mp4
/content/working_dir/61206_20230321_1900downsampled.mp4
Using snapshot-75000 for model /content/drive/MyDrive/DLC/dlc-models/iteration-0/myDLC_modelZooJun3-trainset95shuffle1


/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Starting to analyze %  /content/working_dir/61206_20230321_1900downsampled.mp4
Loading  /content/working_dir/61206_20230321_1900downsampled.mp4
Duration of video [s]:  60.07 , recorded with  15.0 fps!
Overall # of frames:  901  found with (before cropping) frame dimensions:  300 200
Starting to extract posture


100%|██████████| 901/901 [00:09<00:00, 92.93it/s] 


Saving results in /content/csv_dir/...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
/content/csv_dir/61206_20230321_1900downsampledDLC_resnet50_myDLC_modelZooJun3shuffle1_75000.csv
predicting:
/content/csv_dir/61206_20230321_1900downsampledDLC_resnet50_myDLC_modelZooJun3shuffle1_75000.csv
[[ 1.86138611e+02  1.90768661e+02  9.28278327e-01 ...  8.52198486e+01
   3.67050400e+01  2.38369089e-02]
 [ 1.79666229e+02  1.88532883e+02  8.37442577e-01 ...  8.16416473e+01
   4.46908455e+01  6.11048453e-02]
 [ 1.71278275e+02  1.75642868e+02  9.86436665e-01 ...  8.26839523e+01
   4.03038864e+01  3.36977802e-02]
 ...
 [ 1.30266251e+02  1.65141663e+02  6.59518316e-03 ...  1.92057037e+02
   7.44499969e+01  3.67558748e-02]
 [ 1

...

[[6.8644102e-04 7.4724387e-03 9.9184108e-01]]
0.000686441
['/content/csv', 'dir/61206', '20230321', '1900downsampledDLC', 'resnet50', 'myDLC', 'modelZooJun3shuffle1', '75000.csv']
<class 'str'>
csv
2023-03-21 19:09
发现动物正在进食
http://star-eyes.cloud:9090/api/event/addEvent/
10001
{'user_id': '10001', 'password': '00000', 'cam_id': 'csv', 'time': '2023-03-21 19:09', 'event_type': '发现动物正在进食'}
400
{"timestamp":"2023-03-21T11:13:36.699+00:00","status":400,"error":"Bad Request","path":"/api/event/addEvent/"}
Likelihood numpy:
[[0.97177583 0.89304268 0.9152109  ... 0.13860148 0.28022137 0.06777592]
 [0.952694   0.94242764 0.9300577  ... 0.32480252 0.57505041 0.22150618]
 [0.96636045 0.94745648 0.82290608 ... 0.46102929 0.67797238 0.22093241]
 ...
 [0.96560526 0.88874471 0.93462646 ... 0.0647718  0.11467554 0.03144299]
 [0.97169244 0.92631787 0.92992324 ... 0.2921069  0.03308275 0.02869714]
 [0.93497127 0.93122327 0.86040825 ... 0.05761157 0.02089711 0.00887123]]
float64
0.6471699066117355
have 

/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[0.00163567 0.0021358  0.9962286 ]]
0.0016356744
['/content/csv', 'dir/61206', '20230321', '1900downsampledDLC', 'resnet50', 'myDLC', 'modelZooJun3shuffle1', '75000.csv']
<class 'str'>
csv
2023-03-21 19:09
发现动物正在进食
http://star-eyes.cloud:9090/api/event/addEvent/
10001
{'user_id': '10001', 'password': '00000', 'cam_id': 'csv', 'time': '2023-03-21 19:09', 'event_type': '发现动物正在进食'}
400
{"timestamp":"2023-03-21T11:13:45.389+00:00","status":400,"error":"Bad Request","path":"/api/event/addEvent/"}
Likelihood numpy:
[[0.88897264 0.84332097 0.69985038 ... 0.03495375 0.01400063 0.01025559]
 [0.97800702 0.96841383 0.94865918 ... 0.02672198 0.05210698 0.01275883]
 [0.97741276 0.9918555  0.98724771 ... 0.03655219 0.3330946  0.01235777]
 ...
 [0.96570623 0.85842896 0.4792276  ... 0.01104655 0.00699919 0.0228857 ]
 [0.94779634 0.44064665 0.96879554 ... 0.01430755 0.00715481 0.04993565]
 [0.85904133 0.13913403 0.90558845 ... 0.01966079 0.0257945  0.09410346]]
float64
0.319371535574141
have animal, a

/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[0.06562587 0.00261041 0.9317637 ]]
0.06562587
['/content/csv', 'dir/61206', '20230321', '1900downsampledDLC', 'resnet50', 'myDLC', 'modelZooJun3shuffle1', '75000.csv']
<class 'str'>
csv
2023-03-21 19:09
发现动物正在进食
http://star-eyes.cloud:9090/api/event/addEvent/
10001
{'user_id': '10001', 'password': '00000', 'cam_id': 'csv', 'time': '2023-03-21 19:09', 'event_type': '发现动物正在进食'}
400
{"timestamp":"2023-03-21T11:13:54.308+00:00","status":400,"error":"Bad Request","path":"/api/event/addEvent/"}
\D{0}20230321_1910.mp4\D{0}
[<a href="../">../</a>, <a href="61206_20230321_1856.mp4">61206_20230321_1856.mp4</a>, <a href="61206_20230321_1857.mp4">61206_20230321_1857.mp4</a>, <a href="61206_20230321_1858.mp4">61206_20230321_1858.mp4</a>, <a href="61206_20230321_1900.mp4">61206_20230321_1900.mp4</a>, <a href="61206_20230321_1901.mp4">61206_20230321_1901.mp4</a>]
<class 'list'>
[]
Do not find any videos that should be processed at nginx server. This program will do nothing this minute.
\D{0}2023032